In [18]:
import os

os.environ["DEEPSEEK_API_KEY"]='<Your Key>'
os.environ["LLM_MODEL"]='deepseek-chat'

embedding_deployment = "embedding"

In [6]:
pip install -i https://mirrors.aliyun.com/pypi/simple/ chromadb

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org unstructured

  Using cached unstructured-0.16.23-py3-none-any.whl.metadata (24 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 3.0 MB/s eta 0:00:002.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


# 加载文档 （Document Loader）

In [4]:
pip install -i https://mirrors.aliyun.com/pypi/simple/ pypdf

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("./document/Paragraph1.pdf")
docs = loader.load()

# 切割文档
这个文本分割器是通用文本的推荐分割器。它由一个字符列表参数化。它会按顺序尝试在这些字符上进行分割，直到分块足够小。默认列表是["\n\n", "\n", " ", “"]。这样做的效果是尽可能地保持所有段落（然后是句子，然后是单词）在一起。

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 10
)

In [7]:
splits = text_splitter.split_documents(docs)

In [8]:
print(splits[0])

page_content='⻢尔卡斯特（ RichardMulcaster ）就曾写到：   先天禀赋让孩⼦作好准备，后天教养使孩⼦得以成
⻓。   三百年后，⾼尔顿将这⼀对单词变成巧妙、醒⽬的⼴告语，并迅速流⾏开来，成为了我们语
⾔的⼀部分。但教养假设真正的⿐祖是⻄格蒙德   弗洛伊德。弗洛伊德在⼤致的背景下，精⼼地描
绘了这样⼀个场景：即成年⼈的⼼理疾患都可以追溯到年幼时发⽣在他们身上的事情，他们的⽗' metadata={'source': './document/Paragraph1.pdf', 'page': 0, 'page_label': '1'}


In [11]:
persist_directory = 'data/'
# !rm -rf ./data  

# 将数据存入向量数据库

In [14]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

# Ollama local embedding model
ollama_host_url = "http://localhost:11434"
embedding_model = "nomic-embed-text"

vectordb = Chroma.from_documents(
    documents=splits,
    collection_name="education_note",
    embedding=OllamaEmbeddings(model=embedding_model, base_url=ollama_host_url,show_progress=True),
    persist_directory=persist_directory
)
vectordb.persist()
print(vectordb._collection.count())

/var/folders/5v/lmyd52h516jglfw_vqn04sw80000gn/T/ipykernel_16764/3476756762.py:11: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding=OllamaEmbeddings(model=embedding_model, base_url=ollama_host_url,show_progress=True),
OllamaEmbeddings: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  2.92it/s]

5



/var/folders/5v/lmyd52h516jglfw_vqn04sw80000gn/T/ipykernel_16764/3476756762.py:14: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [15]:
docs = vectordb.similarity_search("先天禀赋",k=2)

OllamaEmbeddings: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 30.04it/s]


In [16]:
docs[0]

Document(metadata={'page': 0, 'page_label': '1', 'source': './document/Paragraph1.pdf'}, page_content='⻢尔卡斯特（ RichardMulcaster ）就曾写到： \x00 先天禀赋让孩⼦作好准备，后天教养使孩⼦得以成\n⻓。 \x00 三百年后，⾼尔顿将这⼀对单词变成巧妙、醒⽬的⼴告语，并迅速流⾏开来，成为了我们语\n⾔的⼀部分。但教养假设真正的⿐祖是⻄格蒙德 \x00 弗洛伊德。弗洛伊德在⼤致的背景下，精⼼地描\n绘了这样⼀个场景：即成年⼈的⼼理疾患都可以追溯到年幼时发⽣在他们身上的事情，他们的⽗')

# 利用向量数据库进行QA

https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/retrieval_qa/prompt.py

In [19]:
from langchain.chains import RetrievalQA
from langchain_openai.chat_models.base import BaseChatOpenAI

# Deepseek api
llm = BaseChatOpenAI(
        model=os.getenv("LLM_MODEL"), 
        openai_api_key=os.getenv("DEEPSEEK_API_KEY"), 
        openai_api_base='https://api.deepseek.com',
        max_tokens=1024
    )

retriever=vectordb.as_retriever(
    search_type="mmr", search_kwargs={"k": 3}
    #search_type="similarity", search_kwargs={"k": 3}
)
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="refine", retriever=retriever,
                                 return_source_documents=False, verbose=True)



In [20]:
result = qa({"query": "什么是先天禀赋?"})
print(result["result"])


/var/folders/5v/lmyd52h516jglfw_vqn04sw80000gn/T/ipykernel_16764/2032385179.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query": "什么是先天禀赋?"})




> Entering new RetrievalQA chain...


OllamaEmbeddings: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.80it/s]
Number of requested results 20 is greater than number of elements in index 5, updating n_results = 5



> Finished chain.
根据新的上下文信息，我们可以进一步细化“先天禀赋”的定义。在这个语境中，先天禀赋不仅指孩子与生俱来的天赋或特质，还涉及到孩子在成长过程中所经历的心理发展阶段和危机，这些是由父母（特别是母亲）在孩子早期生活中的角色和行为所引发的。弗洛伊德的理论指出，母亲在孩子断奶和如厕训练等关键发展阶段中扮演了重要角色，这些经历对孩子的心理发展有着深远的影响。因此，先天禀赋在这里也可以理解为孩子在心理发展过程中所固有的、由父母行为所塑造的潜在特质和倾向。这些特质和倾向可能会影响孩子如何应对成长中的挑战和危机。

此外，Judith Harris在《教养的迷思》中提到，在许多社会中，如果孩子的表现太像成年人，往往被认为是没有礼貌的。这表明社会文化背景也会对孩子的行为和表现产生影响，进而影响我们对孩子先天禀赋的理解和评价。因此，先天禀赋不仅包括孩子内在的、与生俱来的特质，还包括这些特质在社会文化环境中的表现和适应方式。


In [21]:
result = qa({"query": "如何定义后天教养？"})
print(result["result"])



> Entering new RetrievalQA chain...


OllamaEmbeddings: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 21.27it/s]
Number of requested results 20 is greater than number of elements in index 5, updating n_results = 5



> Finished chain.
根据新的上下文信息，我们可以进一步细化后天教养的定义。后天教养不仅是指使孩子得以成长的因素，还包括父母在孩子成长过程中所扮演的角色以及他们对孩子心理发展的影响。西格蒙德·弗洛伊德的理论强调了母亲在孩子早期心理发展中的关键作用，尤其是在处理如断奶和如厕训练等基本生活技能时。弗洛伊德认为，这些早期的经历对孩子的心理发展有着深远的影响，并且可能会在成年后引发心理问题。因此，后天教养不仅涉及孩子的成长环境，还包括父母如何引导孩子通过这些关键的成长阶段，以及这些经历如何塑造孩子的心理和行为模式。

此外，新的上下文信息指出，孩子并不是通过简单地模仿父母的行为来学习行为方式。相反，孩子在早期社会化过程中主要学习的是他们不能有像自己父母一样的行为举止。这意味着后天教养还包括父母如何设定界限和规则，以及如何通过禁止某些行为来引导孩子形成适当的行为模式。因此，后天教养不仅涉及父母的行为示范，还包括他们如何通过禁止和引导来塑造孩子的行为和心理发展。
